# Travelling Tourist

I love travelling. And especially going on weekend trips to a new city. With limited time at hand, I want to see as many places as possible.

Unfortunately, these plans usually didn't work out. The planning process is really cumbersome, so I prefered to not consider places to save time.

Many times, I only realized after the trip which mistakes I made, which places I couldn't visit. And how many places I walked buy not knowing about it.

So, ideally, there is a tool that creates me an ideal two-day-route for all the places I want to visit in a city. A tool, that takes into account opening / peak hours of museums. A tool, that optimizes the route so I can make lunch reservation at an amazing restaurant, or visit the foodquarters for an early dinner.

Let's see what is possible!


## Google OR

Ref: https://developers.google.com/optimization/routing

Bascially, a tourist visiting sites in a city is not too different from a delivery person dropping of packages.
And a tourist walking through a city on two separate days, is similar to finding the optimal route for two delivery trucks.

In [3]:
import pandas as pd
import numpy as np

# General preparations
import json
from datetime import datetime, timedelta

# Google Direction API
import googlemaps # Ref: https://www.tomordonez.com/google-maps-api-python/

from mySecrets import GOOGLE_API_DIR

# Google OR
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp

In [4]:
# Setup googlemaps
gmaps = googlemaps.Client(key=GOOGLE_API_DIR)

## A weekend trip to Paris

In [5]:
# Places to visit

class places():

    def __init__(self, name):
        # The name of the trip
        self.name = name
        # The places to visit
        self.places = []
        # The index of the first place, i.e. the depot
        self.hotel_idx = 0
        # The number of days, i.e. the number of vehicles
        self.number_of_days = 2

    # Add new places
    def addPlaces(self, name, address, duration, penalty):
        # duration: time spent at place in seconds
        self.places.append({'name': name, 'address': address, 'duration': duration, 'penalty': penalty})


    # Calculate distance matrix
    def calcDistanceMatrix(self):
        # Number of places to visit, i.e. the dimensions of the matrix
        m_time = []
        m_dist = []
        for i in range(len(self.places)):
            m_time.append([])
            m_dist.append([])
            for j in range(len(self.places)):
                # Diagonal elements = 0
                if i == j:
                    m_time[i].append(0)
                    m_dist[i].append(0)
                # Only fill upper triangular
                elif i > j:
                    m_time[i].append(0)
                    m_dist[i].append(0)
                # All other elements
                else:
                    # Calculate distance
                    directions = gmaps.directions(
                        paris.places[i]['address'], 
                        paris.places[j]['address'], 
                        mode = "walking"
                    )
                    # Distance in meters
                    m_dist[i].append(directions[0]['legs'][0]['distance']['value'])
                    # Distance in seconds
                    m_time[i].append(directions[0]['legs'][0]['duration']['value'])


        # Copy upper triangular to lower triangular
        # Will halve the number of requests
        for i in range(len(paris.places)):
            for j in range(len(paris.places)):
                if i > j:
                    m_time[i][j] = m_time[j][i]
                    m_dist[i][j] = m_dist[j][i]

        self.m_time = m_time
        self.m_dist = m_dist


# Create trip to Paris
paris = places("paris")

# Add the hotel, i.e. your starting and end point each day of the visit
paris.addPlaces('Best Western Premier Faubourg 88', '88 Rue du Faubourg Poissonnière, 75010 Paris, France', 0 * 60, 0)


# Add places to visit ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

# Have to visit!!!!
paris.addPlaces('Tour Eiffel', 'Champ de Mars, 5 Av. Anatole France, 75007 Paris, France', 15 * 60, 5 * 10 ** 6)
paris.addPlaces('Arc de Triomphe', 'Pl. Charles de Gaulle, 75008 Paris, France', 15 * 60, 5 * 10 ** 6)
paris.addPlaces('Sacre Coeur', '35 Rue du Chevalier de la Barre, 75018 Paris, France', 15 * 60, 5 * 10 ** 6)
paris.addPlaces('Notre Dame', '6 Parvis Notre-Dame - Pl. Jean-Paul II, 75004 Paris, France', 15 * 60, 5 * 10 ** 6)
paris.addPlaces('Pantheon', 'Pl. du Panthéon, 75005 Paris, France', 30 * 60, 5 * 10 ** 6)

# Would be nice to visit 
paris.addPlaces('Tour Montparnasse', '33 Av. du Maine, 75015 Paris, France', 30 * 60, 3 * 10 ** 6)
paris.addPlaces('Louvre', 'Rue de Rivoli, 75001 Paris, France', 60 * 60, 3 * 10 ** 6)
paris.addPlaces('Orsay', "1 Rue de la Légion d'Honneur, 75007 Paris, France", 45 * 60, 3 * 10 ** 6)
paris.addPlaces('Rodin', "77 Rue de Varenne, 75007 Paris, France", 45 * 60, 3 * 10 ** 6)
paris.addPlaces('Invalides', "129 Rue de Grenelle, 75007 Paris, France", 30 * 60, 3 * 10 ** 6)
paris.addPlaces('Jardin de Luxembourg', '75006 Paris, France', 30 * 60, 3 * 10 ** 6)
paris.addPlaces("Musee de Picasso", "5 Rue de Thorigny, 75003 Paris, France", 30 * 60, 3 * 10 ** 6)
paris.addPlaces("Parc monceau", "35 Bd de Courcelles, 75008 Paris, France", 30 * 60, 3 * 10 ** 6)

# If time permits
paris.addPlaces("Shakespeare  and company", "37 Rue de la Bûcherie, 75005 Paris, France", 30 * 60, 1 * 10 ** 6)
paris.addPlaces("Place vendome", "Pl. Vendôme, 75001 Paris, France", 15 * 60, 1 * 10 ** 6)
paris.addPlaces("Les halles", "1 Rue Pierre Lescot, 75001 Paris, France", 30 * 60, 1 * 10 ** 6)
paris.addPlaces("Place de vosges", "Pl. des Vosges, 75004 Paris, France", 15 * 60, 1 * 10 ** 6)
paris.addPlaces("Place de la Bastille", "Pl. de la Bastille, 75004 Paris, France", 15 * 60, 1 * 10 ** 6)
paris.addPlaces("Les catacombes", "1 Av. du Colonel Henri Rol-Tanguy, 75014 Paris, France", 60 * 60, 1 * 10 ** 6)
paris.addPlaces("Père Lachaise Cemetery", "16 Rue du Repos, 75020 Paris, France", 30 * 60, 1 * 10 ** 6)




In [6]:
# Print places as markdown table for blog
# Exclude Hotel! As we will vary the lcoation of the hotel.
pd.DataFrame(paris.places[1:]).to_markdown()

"|    | name                     | address                                                     |   duration |   penalty |\n|---:|:-------------------------|:------------------------------------------------------------|-----------:|----------:|\n|  0 | Tour Eiffel              | Champ de Mars, 5 Av. Anatole France, 75007 Paris, France    |        900 |   5000000 |\n|  1 | Arc de Triomphe          | Pl. Charles de Gaulle, 75008 Paris, France                  |        900 |   5000000 |\n|  2 | Sacre Coeur              | 35 Rue du Chevalier de la Barre, 75018 Paris, France        |        900 |   5000000 |\n|  3 | Notre Dame               | 6 Parvis Notre-Dame - Pl. Jean-Paul II, 75004 Paris, France |        900 |   5000000 |\n|  4 | Pantheon                 | Pl. du Panthéon, 75005 Paris, France                        |       1800 |   5000000 |\n|  5 | Tour Montparnasse        | 33 Av. du Maine, 75015 Paris, France                        |       1800 |   3000000 |\n|  6 | Louvre          

In [7]:
# Add distance matrix
paris.calcDistanceMatrix()

In [8]:
# Calculate total time spent at places
sum([i['duration'] for i in paris.places]) / 60 / 60

9.75

In [9]:
# Define printing function - quick overview of calculated optimal solution
def print_solution(places, manager, routing, solution):
    """Prints solution on console."""
    print(f'Objective: {solution.ObjectiveValue()}')
    max_route_distance = 0
    for vehicle_id in range(places.number_of_days):
        index = routing.Start(vehicle_id)
        plan_output = 'Route for day {}:\n'.format(vehicle_id + 1)
        route_distance = 0
        while not routing.IsEnd(index):
            plan_output += ' {} -> '.format(manager.IndexToNode(index))
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(
                previous_index, index, vehicle_id)
        plan_output += '{}\n'.format(manager.IndexToNode(index))
        plan_output += 'Distance of the route: {}seconds\n'.format(route_distance)
        print(plan_output)
        max_route_distance = max(route_distance, max_route_distance)
    print('Maximum of the route distances: {}seconds'.format(max_route_distance))



In [10]:
# Setting up the routing model
manager = pywrapcp.RoutingIndexManager(len(paris.m_time), 
                                       paris.number_of_days, 
                                       paris.hotel_idx)
routing = pywrapcp.RoutingModel(manager)

In [11]:
def time_callback(from_index, to_index):
    """Returns the time it takes to walk between the two nodes."""
    # Convert from routing variable Index to distance matrix NodeIndex.
    from_node = manager.IndexToNode(from_index)
    to_node = manager.IndexToNode(to_index)

    time_to_transit = paris.m_time[from_node][to_node]

    # Add time spent at the location
    time_spent_at_location = paris.places[from_node]['duration']

    # Calc total time
    time_total = time_to_transit + time_spent_at_location
    # time_total = time_to_transit

    return time_total


def distance_callback(from_index, to_index):
    """Returns the distance between the two nodes."""
    # Convert from routing variable Index to distance matrix NodeIndex.
    from_node = manager.IndexToNode(from_index)
    to_node = manager.IndexToNode(to_index)
    return paris.m_dist[from_node][to_node]


# Define callbacks for time and distance
transit_callback_time = routing.RegisterTransitCallback(time_callback)
transit_callback_dist = routing.RegisterTransitCallback(distance_callback)

# Main objective is minimization of time
routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_time)

# For both time and distance, there is a retriction of maximum allowed values

# Per each day, we can walk a maximum of 8 hours
dimension_name = 'MaxTimeSpentWalking'
routing.AddDimension(
    transit_callback_time,
    0,  # no slack
    8 * 60 * 60,  # maximum time available per day: do not want to walk more than 8hours a day
    True,  # start cumul to zero
    dimension_name)
distance_dimension = routing.GetDimensionOrDie(dimension_name)
distance_dimension.SetGlobalSpanCostCoefficient(100)


# Per each day, we can walk a maximum of 15km
dimension_name = 'MaxDistanceWalked'
routing.AddDimension(
    transit_callback_dist,
    0,  # no slack
    20000,  # 
    True,  # start cumul to zero
    dimension_name)
distance_dimension = routing.GetDimensionOrDie(dimension_name)
distance_dimension.SetGlobalSpanCostCoefficient(100)

# Add penalty for dropping / skipping a visit
# TODO: introuce place-specific penalties. Cannot skip eiffel tour, but can skip shakespare and company.
penalty = 10**6
for node in range(1, len(paris.m_time)):
    routing.AddDisjunction([manager.NodeToIndex(node)], penalty)


# Setting first solution heuristic.
search_parameters = pywrapcp.DefaultRoutingSearchParameters()
search_parameters.first_solution_strategy = (
    routing_enums_pb2.FirstSolutionStrategy.AUTOMATIC)

search_parameters.time_limit.seconds = 60

# Solve the problem.
solution = routing.SolveWithParameters(search_parameters)

# Print solution on console.
if solution:
    print_solution(paris, manager, routing, solution)
else:
    print('No solution found !')


Objective: 6538372
Route for day 1:
 0 ->  2 ->  1 ->  10 ->  9 ->  6 ->  11 ->  5 ->  14 ->  16 -> 0
Distance of the route: 28128seconds

Route for day 2:
 0 ->  3 ->  13 ->  15 ->  8 ->  7 ->  4 ->  18 ->  17 ->  12 -> 0
Distance of the route: 27244seconds

Maximum of the route distances: 28128seconds


In [12]:
# Collect all information in dataframe

# Collect places visited
places_output = []
row = 0
for day_id in range(paris.number_of_days):
    index = routing.Start(day_id)
    places_output.append([])
    # route_distance = 0
    while not routing.IsEnd(index):
        places_output[day_id].append(manager.IndexToNode(index))
        previous_index = index
        index = solution.Value(routing.NextVar(index))
    places_output[day_id].append(manager.IndexToNode(index))

# df_places = pd.DataFrame(data={'ID': reduce(concat, places_output)})
df_places = pd.DataFrame(data={'ID': places_output[0] + places_output[1]})


# Add days
df_places['Day'] = 1
number_of_places_per_day = [len(i) for i in places_output]

df_places.loc[number_of_places_per_day[0]:,'Day'] = 2

# Map place id to names and duration
df_places = (
    df_places
    .join(
        pd.DataFrame([{'ID':i, 'Name':values['name'], 'Duration':values['duration']} for i, values in enumerate(paris.places)])
        .set_index('ID'),
        on='ID'
    )
)

# Add transit time to current place
df_places['Transit'] = 0

for i in range(len(df_places)):
    if i != 0:
        idx = df_places.loc[i,'ID']
        idx_prev = df_places.loc[i-1,'ID']
        df_places.loc[i, 'Transit'] = paris.m_time[idx][idx_prev]

# Calculate time of arrival and departure
start_time = datetime(2022, 4, 15, 8, 0, 0)

df_places['time_of_arrival'] = start_time
df_places['time_of_departure'] = start_time

for i in range(len(df_places)):
    if i != 0:
        if df_places.loc[i,'Day'] == df_places.loc[i-1,'Day']:
            df_places.loc[i,'time_of_arrival'] = df_places.loc[i-1,'time_of_departure'] + timedelta(seconds= int(df_places.loc[i,'Transit']))
            df_places.loc[i,'time_of_departure'] = df_places.loc[i,'time_of_arrival'] + timedelta(seconds= int(df_places.loc[i,'Duration']))

# Format to make more readable
df_places['time_of_arrival'] = df_places['time_of_arrival'].dt.strftime("%H:%M")   
df_places['time_of_departure'] = df_places['time_of_departure'].dt.strftime("%H:%M")   


df_places

,ID,Day,Name,Duration,Transit,time_of_arrival,time_of_departure
0,0,1,Best Western Premier Faubourg 88,0,0,08:00,08:00
1,2,1,Arc de Triomphe,900,3214,08:53,09:08
2,1,1,Tour Eiffel,900,1759,09:37,09:52
3,10,1,Invalides,1800,1213,10:13,10:43
4,9,1,Rodin,2700,337,10:48,11:33
5,6,1,Tour Montparnasse,1800,1408,11:57,12:27
6,11,1,Jardin de Luxembourg,1800,1150,12:46,13:16
7,5,1,Pantheon,1800,693,13:27,13:57
8,14,1,Shakespeare and company,1800,688,14:09,14:39
9,16,1,Les halles,1800,897,14:54,15:24


In [13]:
# List of places that you cannot visit :(
places = pd.DataFrame([{'ID':i, 'Name':values['name']} for i, values in enumerate(paris.places)])
places[~places.ID.isin(df_places.ID.unique())]


,ID,Name
19,19,Les catacombes
20,20,Père Lachaise Cemetery


In [19]:
df_places[['Day', 'Name', 'Duration', 'Transit', 'time_of_arrival', 'time_of_departure']].to_markdown()

'|    |   Day | Name                             |   Duration |   Transit | time_of_arrival   | time_of_departure   |\n|---:|------:|:---------------------------------|-----------:|----------:|:------------------|:--------------------|\n|  0 |     1 | Best Western Premier Faubourg 88 |          0 |         0 | 08:00             | 08:00               |\n|  1 |     1 | Arc de Triomphe                  |        900 |      3214 | 08:53             | 09:08               |\n|  2 |     1 | Tour Eiffel                      |        900 |      1759 | 09:37             | 09:52               |\n|  3 |     1 | Invalides                        |       1800 |      1213 | 10:13             | 10:43               |\n|  4 |     1 | Rodin                            |       2700 |       337 | 10:48             | 11:33               |\n|  5 |     1 | Tour Montparnasse                |       1800 |      1408 | 11:57             | 12:27               |\n|  6 |     1 | Jardin de Luxembourg             |      

In [14]:
# Options for restaurants for lunch / dinner at places where we are going to be around lunch time!!
# Inlcude 2 hour lunch + 2 hour dinner

df_places['time_to_lunch'] = [abs(i.total_seconds()) for i in pd.to_datetime(df_places['time_of_departure'], format='%H:%M') - pd.to_datetime('12:00', format='%H:%M')]

df_places


,ID,Day,Name,Duration,Transit,time_of_arrival,time_of_departure,time_to_lunch
0,0,1,Best Western Premier Faubourg 88,0,0,08:00,08:00,14400.0
1,2,1,Arc de Triomphe,900,3214,08:53,09:08,10320.0
2,1,1,Tour Eiffel,900,1759,09:37,09:52,7680.0
3,10,1,Invalides,1800,1213,10:13,10:43,4620.0
4,9,1,Rodin,2700,337,10:48,11:33,1620.0
5,6,1,Tour Montparnasse,1800,1408,11:57,12:27,1620.0
6,11,1,Jardin de Luxembourg,1800,1150,12:46,13:16,4560.0
7,5,1,Pantheon,1800,693,13:27,13:57,7020.0
8,14,1,Shakespeare and company,1800,688,14:09,14:39,9540.0
9,16,1,Les halles,1800,897,14:54,15:24,12240.0


In [15]:
# # Find coordinates or place id of place for lunc at first day

# df_places_lunch = df_places.loc[df_places.Day == 1]
# lunch_location = paris.places[df_places_lunch.time_to_lunch.idxmin()]

# lunch_location_details = gmaps.find_place(lunch_location['address'], 'textquery',fields=['place_id','formatted_address', 'geometry'])

# lat_long = lunch_location_details['candidates'][0]['geometry']['location']

# lunch_places = gmaps.places_nearby(location=(lat_long['lat'], lat_long['lng']), radius=250, keyword="french restaurant", language="en")


# lunch_day_one = (
#     pd.DataFrame([{'Name': item['name'], 'Rating': item['rating'], 'N_Ratings': item['user_ratings_total'], 'Vicinity': item['vicinity']} for item in lunch_places['results']])
#     .sort_values('Rating', ascending=False)
#     # Min number of ratings
#     .query('N_Ratings > 50')
#     # Min number of rating
#     .query('Rating > 3.8')
# )

In [16]:
# Use folium to customize map

# Put markers for both days with different colors
# Mark the hotel with a different marker!
# Zoom to all inner-paris -> including Neuilly / La Defense?

import folium
# https://coderzcolumn.com/tutorials/data-science/interactive-maps-choropleth-scattermap-using-folium

# Center map at Louvre
center_lat = 48.862983
center_long = 2.3202568

m = folium.Map(location=[center_lat, center_long], zoom_start=13.5)#, tiles="Stamen Terrain")

# Find Geocodes for all places

addresses = [p['address'] for p in paris.places]


geocodes = [gmaps.geocode(address=a)[0]['geometry']['location'] for a in addresses]

loc_1 = df_places.loc[df_places.Day == 1]['ID']
loc_2 = df_places.loc[df_places.Day == 2]['ID']

geocodes_1 = [[geocodes[i]['lat'], geocodes[i]['lng']] for i in loc_1] 
geocodes_2 = [[geocodes[i]['lat'], geocodes[i]['lng']] for i in loc_2] 

# Day 1 ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# geocodes[0]['lat']
for i in loc_1:
    folium.Marker(
    location=[geocodes[i]['lat'], geocodes[i]['lng']],
    popup="Test",
    # tooltip = district_census.District.values[0],
    icon=folium.Icon(icon='camera', color="orange")
    ).add_to(m)

folium.PolyLine(
    locations=geocodes_1,
    popup="Path",
    tooltip = "Path",
    color="orange",
).add_to(m)

# Day 2 ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
for i in loc_2:
    folium.Marker(
    location=[geocodes[i]['lat'], geocodes[i]['lng']],
    popup="Test",
    # tooltip = district_census.District.values[0],
    icon=folium.Icon(icon='camera', color="blue")
    ).add_to(m)

folium.PolyLine(
    locations=geocodes_2,
    popup="Path",
    tooltip = "Path",
    color="blue",
).add_to(m)

# Mark hotel ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
folium.Marker(
    location=[geocodes[0]['lat'], geocodes[0]['lng']],
    popup="Test",
    # tooltip = district_census.District.values[0],
    icon=folium.Icon(icon='home', color="red")
    ).add_to(m)


# Display map
m

# Save
m.save("graphics\Paris.html")

In [17]:
# Hack to save folium maps as png

import os
import time
from selenium import webdriver

delay=5
fn='graphics\Paris.html'
tmpurl='{path}\{mapfile}'.format(path=os.getcwd(),mapfile=fn)
# m.save(fn)

browser = webdriver.Firefox()
browser.get(tmpurl)
#Give the map tiles some time to load
time.sleep(delay)
browser.save_screenshot('graphics\Paris.png')
browser.quit()

# REduce png with https://ezgif.com/optipng